In [ ]:
%cd ../

In [1]:
from extensions.encoder_decoder_correlations import EncoderDecoderComparison
import torch

from captum.attr import GradientShap, IntegratedGradients, Saliency

/home/laurens/Desktop/FACT/Label-Free-XAI/my_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define globals
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Gradshap

In [3]:
gradient_shap_experiment = EncoderDecoderComparison(
    model_name="",
    attributer_factory=GradientShap,
    dataset='MNIST',
    device = DEVICE,
)

Loading data...
Loading Models and Calculating Attributions...
Loading model in ../TrainedModels/MNIST/Classifier_run1.pt
Loading model in ../TrainedModels/MNIST/Classifier_run0.pt
Loading model in ../TrainedModels/MNIST/Classifier_run2.pt
Loading model in ../TrainedModels/MNIST/Classifier_run3.pt
Loading model in ../TrainedModels/MNIST/Classifier_run4.pt
Complete


In [4]:
res = gradient_shap_experiment.get_all_model_pearsons(True)

In [5]:
print(res.mean(axis=1))



[0.3503069  0.32580192 0.32778726 0.36071626 0.32253383]


## Integrated Gradients

In [6]:
ig_experiment = EncoderDecoderComparison(
    model_name="",
    attributer_factory=IntegratedGradients,
    dataset='MNIST',
    device = DEVICE,
)

Loading data...
Loading Models and Calculating Attributions...
Loading model in ../TrainedModels/MNIST/Classifier_run1.pt
Loading model in ../TrainedModels/MNIST/Classifier_run0.pt
Loading model in ../TrainedModels/MNIST/Classifier_run2.pt
Loading model in ../TrainedModels/MNIST/Classifier_run3.pt
Loading model in ../TrainedModels/MNIST/Classifier_run4.pt
Complete


In [7]:
ig_pearsons = ig_experiment.get_all_model_pearsons(True)
ig_pearsons.mean(axis=1)

array([0.39079937, 0.47364313, 0.45926487, 0.42462469, 0.41964185])

## Saliency

In [8]:
sal_experiment = EncoderDecoderComparison(
    model_name="",
    attributer_factory=Saliency,
    dataset='MNIST',
    device = DEVICE,
)

Loading data...
Loading Models and Calculating Attributions...
Loading model in ../TrainedModels/MNIST/Classifier_run1.pt


/home/laurens/Desktop/FACT/Label-Free-XAI/my_env/lib/python3.8/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


Loading model in ../TrainedModels/MNIST/Classifier_run0.pt
Loading model in ../TrainedModels/MNIST/Classifier_run2.pt
Loading model in ../TrainedModels/MNIST/Classifier_run3.pt
Loading model in ../TrainedModels/MNIST/Classifier_run4.pt
Complete


In [9]:
sal_pearsons = sal_experiment.get_all_model_pearsons(True)
sal_pearsons.mean(axis=1)

array([0.21530681, 0.13549975, 0.16179986, 0.16258512, 0.20697079])